In [1]:
import os
import pandas as pd
import sqlite3

In [2]:
# Cell 0: read CSV, drop rows with missing values, save to SQLite

raw_path = "data/raw/NVDA.csv"
processed_dir = "data/processed"
os.makedirs(processed_dir, exist_ok=True)
sqlite_path = os.path.join(processed_dir, "NVDA_proc.sqlite")

# load, drop missing rows, reset index
df = pd.read_csv(raw_path)
df_clean = df.dropna().reset_index(drop=True)

# write to sqlite (table name: NVDA)
with sqlite3.connect(sqlite_path) as conn:
    df_clean.to_sql("NVDA", conn, if_exists="replace", index=False)

print(f"Cleaned data saved to {sqlite_path} ({len(df_clean)} rows).")

Cleaned data saved to data/processed\NVDA_proc.sqlite (6514 rows).
